### Importowanie pakietów z znanym nam zbiorem iris

In [22]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris

In [23]:
iris = load_iris()
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

In [24]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [25]:
# !pip install sqlalchemy

### Tworzenie i łączenie się z bazą danych SQL

Polecenie create_engine pozwala nam się podłączyć do już istniejącej bazy danych.
SQLite to baza danych "plikowa". Stworzy nam się plik .db, który będzie zawierałnasze bazy danych

In [26]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///irysy.db')

Za pomocą polecenia to_sql możemy zapisać nasze obiekty jako tabele sql-owe i zapisać w bazie danych

In [27]:
df.to_sql('dane3', con=engine, index=False)

In [28]:
a = engine.execute("SELECT * FROM dane").fetchall()

OperationalError: (sqlite3.OperationalError) no such table: dane
[SQL: SELECT * FROM dane]
(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [29]:
type(a)

list

In [30]:
type(a[0])

sqlalchemy.engine.row.LegacyRow

Te dane możemy skonwertować do data frame

In [31]:
pd.DataFrame(a)

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [32]:
df2 = pd.DataFrame({'name' : ['User 1', 'User 2', 'User 3']})
df2

,name
0,User 1
1,User 2
2,User 3


In [33]:
df2.to_sql('users', con=engine, if_exists="replace")
engine.execute("SELECT * FROM users").fetchall()

[(0, 'User 1'), (1, 'User 2'), (2, 'User 3')]

In [34]:
df3 = pd.DataFrame({'name' : ['User 6', 'User 7']})
df3.to_sql('users', con=engine, if_exists='append')

In [35]:
engine.execute("SELECT * FROM users").fetchall()

[(0, 'User 1'), (1, 'User 2'), (2, 'User 3'), (0, 'User 6'), (1, 'User 7')]

In [36]:
from sqlalchemy import create_engine
from sqlalchemy import Column, String, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [37]:
engine = create_engine('sqlite:///irysy.db')
base = declarative_base()

In [38]:
class Transakcje(base):
    
    __tablename__ = 'transakcje'
    
    transakcja_id = Column(Integer, primary_key=True )
    data = Column(String)
    przedmiot_id = Column(Integer)
    cena = Column(Integer)
    
    def __init__(self, transakcja_id, data, przedmiot_id, cena):
        self.transakcja_id = transakcja_id
        self.data = data
        self.przedmiot_id = przedmiot_id
        self.cena = cena

In [39]:
class Transakcje2(base):
    
    __tablename__ = 'tracje'
    
    transakcja_id = Column(Integer, primary_key=True )
    data = Column(String)
    przedmiot_id = Column(Integer)
    cena = Column(Integer)
    
    def __init__(self, transakcja_id, data, przedmiot_id, cena):
        self.transakcja_id = transakcja_id
        self.data = data
        self.przedmiot_id = przedmiot_id
        self.cena = cena

In [40]:
# tworzenie tabel
base.metadata.create_all(engine)

In [41]:
from sqlalchemy.orm import sessionmaker
# Stworzenie nowej sesji
Session = sessionmaker(bind=engine)
session = Session()
# dodanie danych
for t in range(10):
    tr = Transakcje(t, f'200{t}/05/06', t**2-t*2, 19)
    session.add(tr)
# zapis zmian w bazie danych
session.commit()

In [42]:
Session = sessionmaker(bind=engine)
session = Session()
# dodanie danych
for t in range(10):
    tr = Transakcje2(t, f'200{t}/05/06', t**2-t*2, 19)
    session.add(tr)
# zapis zmian w bazie danych
session.commit()

In [43]:
engine.execute("SELECT * FROM tracje").fetchall()

[(0, '2000/05/06', 0, 19),
 (1, '2001/05/06', -1, 19),
 (2, '2002/05/06', 0, 19),
 (3, '2003/05/06', 3, 19),
 (4, '2004/05/06', 8, 19),
 (5, '2005/05/06', 15, 19),
 (6, '2006/05/06', 24, 19),
 (7, '2007/05/06', 35, 19),
 (8, '2008/05/06', 48, 19),
 (9, '2009/05/06', 63, 19)]

In [44]:
# wszystkie dane
for s in session.query(Transakcje).all():
    print(s.transakcja_id, s.data)

0 2000/05/06
1 2001/05/06
2 2002/05/06
3 2003/05/06
4 2004/05/06
5 2005/05/06
6 2006/05/06
7 2007/05/06
8 2008/05/06
9 2009/05/06


In [45]:
# wybrane transakcje
for s in session.query(Transakcje).filter(Transakcje.transakcja_id>5):
    print(s.transakcja_id, s.data)

6 2006/05/06
7 2007/05/06
8 2008/05/06
9 2009/05/06


In [46]:
import json

JSON - Java Script Object Notation (ale w żaden sposób nie jest połączony z Java Scriptem czy żadnym innym językiem).  
JSON to sposób na lekki format wymiany danych.  
W JSON są czyste dane, nie ma tam jakichś zmiennych czy fukncji. Popularyazcja jego nastąpiła od mniej więcej 2001. Teraz jest używanie prawie, że domyślnie do wymiany danych między aplikacjami.

In [47]:
from IPython.display import HTML
from IPython.display import Image

In [48]:
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/haYYypSnOTY" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

c:\programdata\miniconda3\lib\site-packages\IPython\core\display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [49]:
person = '{"name": "Bob", "languages": ["English", "Fench"]}'
type(person)

str

In [50]:
person_dict = json.loads(person)

# Output: {'name': 'Bob', 'languages': ['English', 'Fench']}
print(person_dict)

# Output: ['English', 'French']
print(person_dict['languages'])

{'name': 'Bob', 'languages': ['English', 'Fench']}
['English', 'Fench']


In [51]:
type(person_dict)

dict

In [52]:
%%file test.json


{"name": "Bob", 
"languages": ["English", "Fench"]
}

Overwriting test.json


In [53]:
with open('test.json') as f:
    data = json.load(f)

# Output: {'name': 'Bob', 'languages': ['English', 'Fench']}
print(data)

{'name': 'Bob', 'languages': ['English', 'Fench']}


In [54]:
type(data)

dict

In [55]:
person_dict = {'name': 'Bob',
'age': 12,
'children': None
}
person_json = json.dumps(person_dict)

print(person_json)


{"name": "Bob", "age": 12, "children": null}


In [56]:
with open('person.json', 'w') as json_file:
    json.dump(person_dict, json_file)

In [57]:
print(json.dumps(person_dict, indent = 4, sort_keys=True))

{
    "age": 12,
    "children": null,
    "name": "Bob"
}


In [58]:
!pip install flask
!pip install flask_restful
!pip install flask_jsonpify

  Created wheel for flask-jsonpify: filename=Flask_Jsonpify-1.5.0-py3-none-any.whl size=3087 sha256=6527506d64f07be686e50d625d4f9bac8e13254c0c0f7e3cde09eab5c08738d7
  Stored in directory: c:\users\vlad\appdata\local\pip\cache\wheels\59\f3\72\d9f6107c47949953788fd4f531a52b22ec94d864db07ed92d4
Successfully built flask-jsonpify


In [59]:
from flask import Flask, request
from flask_restful import Resource, Api
from flask_jsonpify import jsonify

In [60]:
app  = Flask(__name__)
api = Api(app)

@app.route('/')
@app.route('/index')
def home():
    #return render_template('home.html')
    return "Strona poczatkowa"

@app.route('/hello/<name>')
def success(name):
    return f'<h1>{name}</h1>'

In [61]:
class Main(Resource):
    def get(self):
        return jsonify("Hello world")

In [62]:
api.add_resource(Main,'/test')

In [63]:
class Irys(Resource):
    
    def get(self):
        conn=engine.connect()
        query = conn.execute('select * from dane')
        result = {'dane':[i for i in query.cursor.fetchall()]}
        return jsonify(result)

In [64]:
api.add_resource(Irys, '/irys')

In [ ]:
app.run(port='5002')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5002/ (Press CTRL+C to quit)
